<a href="https://colab.research.google.com/github/UmutMD/Star-Prediction-K-Fold/blob/main/star_prediction_k_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>